In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from absl import app
from absl import flags
import collections

from dm_control import mujoco
from dm_control.rl import control
from dm_control.suite import base
from dm_control.suite import common
from dm_control.suite.utils import randomizers
from dm_control.utils import containers

import scipy.io as sio
import matplotlib
matplotlib.use("Agg")
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np

In [2]:
_DEFAULT_TIME_LIMIT = 20
_CONTROL_SUBSTEPS = 100 #for use with control.Environment(... n_sub_steps=#)
CONVERSION_LENGTH = 1000 #ASSUMES .MAT TO BE IN MILLIMETERS

fileName='.\\dataInput\\ratMocapImputedUnregisteredDataIn_2'
varName='markers_preproc'
max_frame=33000
start_frame=24000
frame_step=100
model_filename='ratMocap\\models\\rodent_mocap_tendon_def2.xml'

In [3]:
def get_model_and_assets():
  """Returns a tuple containing the model XML string and a dict of assets."""
  return common.read_model(model_filename), common.ASSETS

In [4]:
class jeffRat(base.Task):
    def __init__(self, random=None):
        super(jeffRat, self).__init__(random=random)
    def initialize_episode(self, physics):
        penetrating = True
        while penetrating:
            randomizers.randomize_limited_and_rotational_joints(
                physics, self.random)
            physics.after_reset()
            penetrating = physics.data.ncon > 0
    def get_observation(self, physics):
        obs = collections.OrderedDict()
        obs['joint_angles'] = physics.joint_angles()
        return obs
    def mocap(time_limit=_DEFAULT_TIME_LIMIT, random=None, environment_kwargs=None):
        physics = mujoco.Physics.from_xml_string(*get_model_and_assets())
        task = jeffRat(random=random)
        environment_kwargs = environment_kwargs or {}
        return control.Environment(
            physics, task, time_limit=time_limit, n_sub_steps=_CONTROL_SUBSTEPS,
            **environment_kwargs)
    def get_reward(self, physics):
        return 0

In [5]:
def parse(fileName, varName):
  parsed = collections.namedtuple('parsed', ['marks', 'bods', 'medianPose', 'mocap_pos'])
  marks = dict()
  medianPose = dict()
  zOffset = dict()
  values = sio.loadmat(fileName, variable_names = varName)
  bods = list(values[varName].dtype.fields)
  for bod in bods:
    marks[bod] = values[varName][bod][0][0]/CONVERSION_LENGTH #ASSUMES .MAT TO BE IN MILLIMETERS
    zOffset[bod] = np.amin(marks[bod], axis = 0) * [0, 0, 1]
  if min(zOffset[min(zOffset)]) < 0:
    for bod in bods:
      marks[bod] += abs(zOffset[min(zOffset)])
  for bod in bods:
    if bod == bods[0]:
      mocap_pos = marks[bod]
    else:
      mocap_pos = np.concatenate((mocap_pos, marks[bod]), axis = 1)
    medianPose[bod] = np.median(marks[bod], axis = {0})
  mocap_pos.shape = (mocap_pos.shape[0], int(mocap_pos.shape[1]/3), 3)
  mocap_pos = forFillNan(mocap_pos)
  return parsed(marks, bods, medianPose, mocap_pos)

In [6]:
def forFillNan(arr):
    for m in range(arr.shape[1]):
        if any(np.isnan(arr[:, m, 1])):
            ind = np.where(np.isnan(arr[:, m, 1])) #only works if all coordinates are NaNs    
            for i in ind[0]:
                if i == 0:
                    arr[i, m, :] = [0, 0, 0]
                else:
                    arr[i, m, :] = arr[i - 1, m, :].copy()
    return arr

In [7]:
def showFrame(data, frame, i):
    p_i = data.mocap_pos[i, :].copy()
    with env.physics.reset_context():
        env.physics.data.mocap_pos[:] = p_i
    while env.physics.time() < 2. or np.nanmean(abs(env.physics.data.qvel)) > 1e-06:
        env.physics.step()
    frame = np.hstack([env.physics.render(height, width, camera_id="front_side")])
    img = plt.imshow(frame)
    plt.waitforbuttonpress()
    plt.close()

In [8]:
def getFrame(data, i):
    p_i = data.mocap_pos[i, :].copy()
    with env.physics.reset_context():
        env.physics.data.mocap_pos[:] = p_i
    while env.physics.time() < 2. or np.nanmean(abs(env.physics.data.qvel)) > 1e-06:
        env.physics.step()
    return np.hstack([env.physics.render(height, width, camera_id="front_side")])    

In [9]:
env = jeffRat.mocap()
data = parse(fileName, varName)
metadata = dict(title= model_filename + ': ' + fileName, artist='Jeff Rhoades/Jesse Marshall/DeepMind',
                  comment=varName)
writer = animation.FFMpegWriter(fps=3, metadata=metadata)
width = 500
height = 500

In [10]:
max_frame = min(max_frame, data.mocap_pos.shape[0])
max_num_frames = (max_frame - start_frame)//frame_step
video = np.zeros((max_num_frames, height, width, 3), dtype=np.uint8)
qVid = np.zeros((max_num_frames, env.physics.data.qpos.size), dtype=np.uint8)

In [11]:
with env.physics.reset_context():
      env.physics.data.qpos[:] = env.physics.model.key_qpos

In [12]:
for i in range(start_frame, max_frame, frame_step):
    p_i = data.mocap_pos[i, :].copy()
    with env.physics.reset_context():
      env.physics.data.mocap_pos[:] = p_i
    while env.physics.time() < 2. or np.nanmean(abs(env.physics.data.qvel)) > 1e-06:
      env.physics.step()

    video[(i - start_frame)//frame_step] = np.hstack([env.physics.render(height, width, camera_id="front_side")])
    qVid[(i - start_frame)//frame_step] = env.physics.data.qpos[:]

In [13]:
fig = plt.figure()
img = plt.imshow(video[1])

In [14]:
plt.rcParams['animation.ffmpeg_path'] = r'C:\Users\RatControl\ffmpeg\bin\ffmpeg.exe'

In [15]:
with writer.saving(fig, fileName + "_vid.mp4", dpi=None):
    for i in range(video.shape[0]):
      if i == 0:
        img = plt.imshow(video[i])
      else:
        img.set_data(video[i])
      plt.draw()
      writer.grab_frame()
writer.finish()

In [16]:
env.physics.data.mocap_pos

array([[-0.20050879, -0.01889014,  0.21704356],
       [-0.18220117, -0.01266324,  0.22032623],
       [-0.17054621, -0.03844966,  0.21035116],
       [-0.1333779 ,  0.01014074,  0.1618303 ],
       [-0.10832166,  0.01087516,  0.13484954],
       [-0.08361758, -0.00404689,  0.09187872],
       [-0.1190781 , -0.01097721,  0.14232928],
       [-0.10013303, -0.01918055,  0.1170243 ],
       [-0.08799545, -0.03027704,  0.08260977],
       [-0.0912778 ,  0.01735471,  0.06860284],
       [-0.15648688, -0.02727387,  0.12760641],
       [-0.16698886, -0.03060118,  0.12748726],
       [-0.13816099, -0.0096489 ,  0.15634668],
       [-0.15239352,  0.02373856,  0.15573161],
       [-0.18088241,  0.02068534,  0.13761258],
       [-0.19555689,  0.01331954,  0.15268864],
       [-0.11303228,  0.0260468 ,  0.06954454],
       [-0.10713802, -0.0367506 ,  0.07903204],
       [-0.10983214, -0.03521154,  0.05566167],
       [-0.11018158,  0.02075251,  0.04510733]])

In [17]:
def forFillNan(arr):
    for m in range(arr.shape[0]):
        if any(np.isnan(arr[m, :])):
            ind = np.where(np.isnan(arr[m, :])) #only works if all coordinates are NaNs    
            for i in ind[0]:
                if i == 0:
                    arr[i, m, :] = [0, 0, 0]
                else:
                    arr[i, m, :] = arr[i - 1, m, :].copy()
    return arr

In [18]:
for bod in data.bods:
    data.marks[bod] = forFillNan(data.marks[bod])

In [19]:
bod = data.bods[0]

In [20]:
data.marks[bod]

array([[ 0.1962218 , -0.20207783,  0.12215507],
       [ 0.19646578, -0.20028693,  0.13836126],
       [ 0.19658452, -0.19864575,  0.15362475],
       ...,
       [ 0.17846843,  0.02026208,  0.0970722 ],
       [ 0.17874677,  0.0200602 ,  0.09720005],
       [ 0.17904574,  0.01983942,  0.0972932 ]])

In [46]:
physics = env.physics
qpos = physics.data.qpos[:2]
print(qpos)

[-9.26438875e-02  8.91518317e-05]


In [197]:
goalAngles = np.array([[30, 90], [45, 30], [90, 45]])

In [292]:
for i in goalAngles:
    if not all(i == goalAngles[-1]):
        print('yeah')
    else:
        print('no')

yeah
yeah
no


In [298]:
1*(dOut == 0) - 1*(dOut > 0)

array([ 1, -1,  1, -1,  1, -1])

In [75]:
vec = np.array([[1, 0], [-1, 0], [0, 1], [0, -1], [1, 1], [-1, -1]])
oldDeltaQ = np.zeros(qpos.shape)
newDeltaQ = qpos[:2] - i

In [188]:
dOut = np.sum(vec*newDeltaQ, 1)
dOut = dOut*(dOut > 0)
dDeltaQ = newDeltaQ - oldDeltaQ

alphaNorm = np.max(abs(np.degrees(physics.model.jnt_range[:2])), 1)
alphaNorm = alphaNorm + 1*(alphaNorm == 0)
alpha = np.sum(vec*dDeltaQ/alphaNorm, 1)
alpha = 1 + alpha*(alpha < 0)

In [190]:
alphaNorm

array([  1., 180.])

In [157]:
np.max(abs(np.degrees(physics.model.jnt_range[1:3])), 1)

array([180., 180.])

In [158]:
np.degrees(physics.model.jnt_range[:3])

array([[   0.,    0.],
       [-180.,  180.],
       [-180.,  180.]])

In [168]:
out = dOut + alpha*out

array([  0.        ,  90.18528778,   0.        ,  59.9998217 ,
         0.        , 150.18510947])

0.7333333333333334

In [272]:
newDeltaQ

array([-45.09264389, -29.99991085])

In [174]:
import collections
handle = collections.namedtuple('handle', ['a', 'b', 'c', 'd'])
h = handle(1, [], 3, [])
h.b = 2

AttributeError: can't set attribute

In [267]:
anti = out*(dOut > 0)
anti

array([  0.        ,  90.18528778,   0.        ,  59.9998217 ,
         0.        , 150.18510947])

In [212]:
vec

array([[ 1,  0],
       [-1,  0],
       [ 0,  1],
       [ 0, -1],
       [ 1,  1],
       [-1, -1]])

In [456]:
anti = np.sum(anti*(anti < 0),1)
anti

array([-240.37039725,    0.        ,   -1.16769406,    0.        ,
       -241.53809131,    0.        ])

In [275]:
np.sum(vec*newDeltaQ, 1)

array([-45.09264389,  45.09264389, -29.99991085,  29.99991085,
       -75.09255474,  75.09255474])

In [311]:
fileName = r'C:\Users\RatControl\Anaconda3\Lib\site-packages\dm_control\suite\ratMocap\dataInput\nolj_Recording_day3_morning2_nolj_imputed_JDM25_scaled.mat'
import os
from pathlib import Path
inFile = Path(fileName).absolute()

In [450]:
anti*vec == vec*anti

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])

In [488]:
env.physics.model.labels

AttributeError: 'MjModel' object has no attribute 'labels'

In [360]:
(dOut > 0)*(vec*newDeltaQ/alphaNorm).transpose()

array([[-0.        , 45.09264389, -0.        , -0.        , -0.        ,
        45.09264389],
       [-0.        , -0.        , -0.        ,  0.16666617, -0.        ,
         0.16666617]])

In [424]:
anti = anti+[[1, 0, 0, 0, 0, 0],[0, 1, 0, 0, 0, 0]]
np.sum(anti, 1)

ValueError: operands could not be broadcast together with shapes (2,) (2,6) 

In [423]:
anti = np.sum((dOut > 0)*(vec*newDeltaQ/alphaNorm).transpose(), 1)
np.sum(anti*(vec > 0), 1)

array([90.18528778,  0.        ,  0.33333234,  0.        , 90.51862012,
        0.        ])

In [379]:
ext = np.sum(((dOut >= 0)*out*(vec < 0).transpose()), 1)

array([[   0.        ,  -90.18528778,    0.        ,    0.        ,
           0.        , -150.18510947],
       [   0.        ,    0.        ,    0.        ,  -59.9998217 ,
           0.        , -150.18510947]])

In [455]:
anti = vec*np.sum(((dOut >= 0)*out*vec.transpose()), 1)/alphaNorm

In [9]:
import numpy as np, h5py 
f = h5py.File('.\\dataInput\\mujocosnippets_Baseline1_JDM27.mat','r') 
data = f.get('data/variable1') # Get a certain dataset
data = np.array(data)

OSError: Unable to open file (file signature not found)

In [13]:
values = sio.loadmat('.\\dataInput\\mujocosnippets_Baseline1_JDM27.mat')
values

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Feb 20 16:10:10 2019',
 '__version__': '1.0',
 '__globals__': [],
 'snippetstruct': array([[array([[(array([[(array([[ 60.63460101, -83.1779571 , -32.91431685],
        [ 59.89852063, -83.13460083, -33.5457864 ],
        [ 58.84800805, -83.21417593, -33.89247092],
        ...,
        [ 66.4121043 , -84.79667394, -28.84245436],
        [ 65.54682792, -84.53425038, -28.3169418 ],
        [ 64.50958106, -83.63275504, -28.15147613]]), array([[ 73.97192526, -78.75063269, -18.95918545],
        [ 73.29588041, -78.96748957, -19.57123608],
        [ 72.29964469, -79.33675838, -19.76241341],
        ...,
        [ 77.07763077, -77.38268412, -13.78927123],
        [ 76.27362836, -77.63390009, -13.13447055],
        [ 75.35020172, -77.69636224, -12.71436606]]), array([[ 90.51282278, -62.2532024 , -37.82068744],
        [ 89.94373637, -62.48551537, -38.32719734],
        [ 89.45417323, -63.14348568, -38.2992587 ],
        ..

In [67]:
values['snippetstruct'][0, 88].dtype.fields

mappingproxy({'aligned_mocap': (dtype('O'), 0),
              'agg_preproc': (dtype('O'), 8)})

In [1]:
import matlab.engine

ModuleNotFoundError: No module named 'matlab.engine'; 'matlab' is not a package